# Word Embeding



对于一个有$V$ 个词的词库（vocabulary），我们总可以定义$V$ 维的one-hot 矢量表征每个词，但当$V\gg1$时，这样的表征太过“稀疏”，且完全无法体现词语间关系。我们希望通过某种合理手段，将表征词的embedding空间降为$D$。 因此word embedding 方法可以看做是类似于t-SNE的降维手段。具体操作步骤如下：

(1). 设语料库corpus中总共有$V$个独立词，（即$size(\text{Vocabulary}) = V$,一般将词频很少的罕见词都合并成一个 UNKNOWN 词看待），并根据一定标准（通常为词频大小排序）将这些词用整数 $0,1,\dots,V-1$ 来编号，例如：

$$\text{UNKNOWN $\rightarrow 0, \;$ "the" $\rightarrow 1, \;$ "of" $\rightarrow 2, \;$ "I" $\rightarrow 3$, $\dots$}$$

根据这个映射关系，可以将corpus看做一个整数序列，例如：

$$\begin{matrix}
\dots&\text{fox}&\text{jumped}&\text{over}&\text{the}&\text{lazy}&\text{dog}&\dots\\
&\uparrow&\uparrow&\uparrow&\uparrow&\uparrow&\uparrow&\\
[\dots&235,&421,&13,&1,&32,&79,&\dots]\end{matrix}$$

(2). 选定embedding 空间的维度$D$，即每个词将用一个$D$维向量表征，并随机初始化一个$V\times D$ 维的矩阵 $W$, $W$ 的第$i$行将用来表示编号为$i$ 的词, 例如

$$W\equiv\begin{bmatrix}W_0\\W_1\\\vdots\\W_{V-1}\end{bmatrix}
\begin{matrix}\longrightarrow& w_0 & = &\text{UNKNOWN}\\\longrightarrow& w_1 &= &\text{"the"}\\\vdots\\\longrightarrow& w_{V-1}& =\end{matrix}
\quad W_i= \underbrace{[\dots\dots]}_{D \text{维向量}}$$

(3). 选定一个窗口尺寸（例如5），让窗口在corpus 上连续“滑动”，窗口每“滑动”一步，截取个数为窗口尺寸（例如5）的一组词（例如: fox(235) jumped(421) over(13) the(1) lazy(32) ），并将这组词中间的那个（例如: over）, 称为target，记为$t$，剩余的词（例如: [fox, jumped, the, lazy]）称为context，记为$c$。设总共可以截取$N$组这样的词， 每组称为一个sample。整个sample集记为 $\mathcal{D}=\{d^{(\alpha)}\equiv(c^{(\alpha)}, t^{(\alpha)}),\; \alpha=1,2,\dots,N\}$, (其中 $c^{(\alpha)}$ 是sample $\alpha$ 的context，$t^{(\alpha)}$是sample $\alpha$ 的target) 

某种程度上可以说，我们通过窗口采样获得训练集，通过context和target的划分，我们将一个<strong style="color:red">unsupervised learning 问题“转化为”supervised learning问题来处理了</strong>


(4). 后文为叙述方便，不妨设窗口尺寸为5,对于 $\mathcal{D}$中某个sample $\alpha$， 记target of sample $\alpha$为 $t^{(\alpha)}=w_p$, context of sample $\alpha$ 为 $c^{(\alpha)}=[w_k, w_l, w_m, w_n]$，我们认为 描述context $c^{(\alpha)}$ 的 输入量可以取为context中所有词的“重心”：
$$C^{(\alpha)} \equiv {W_k + W_l + W_m + W_n\over 4},\quad\text{显然这种取法中，$C^{(\alpha)} $ 也是$D$ 维向量，且context内的词序信息丢失}$$

(5-1). 根据softmax思想，对 $\forall$ sample $\alpha$，定义$V$ 维的归一化矢量：

$$\hat{y}^{(\alpha)}={1\over \sum\limits_{i} \exp\left(C^{(\alpha)}\cdot W_i\right)}\begin{bmatrix}
\exp\left(C^{(\alpha)}\cdot W_0\right)\\
\exp\left(C^{(\alpha)}\cdot W_1\right)\\
\vdots\\
\exp\left(C^{(\alpha)}\cdot W_{V-1}\right)\\
\end{bmatrix}\quad\text{用来描述 target of sample $\alpha$ 的在vocabulary上的概率预期}$$

根据 target of sample $\alpha$的经验值, $t^{(\alpha)}=w_p$ ，我们可以定义$V$ 维的one-hot矢量

$$ \begin{matrix}
&\text{第0个}&&\text{第$p-1$个}&&\text{第$V-1$个}&\\
&\downarrow&&\downarrow&&\downarrow&\\
y^{(\alpha)} = [&0&,\dots,0,&1&,0,\dots,&0&]\\
\end{matrix}$$

相应的cross-entropy函数为

$$\mathscr{C}^{(\alpha)}=-y^{(\alpha)}\cdot \log \hat{y}^{(\alpha)} $$

于是整个样本集$\mathcal{D}$ 的cost function 为：

$$\mathscr{C} = {1\over N}\sum\limits_{\alpha=1}^N\mathscr{C}^{(\alpha)}$$

vocabulary 中每个词对应的 embedding vector为：

$$\left(\;W_0^*, W_1^*,\dots, W_{V-1}^*\;\right) ={\rm argmin} \mathscr{C}\left(\;W_0, W_1,\dots, W_{V-1}\;\right)$$

(5-2). 上节给出的方法，原则上可以给出合理的embedding vector，但由于$\hat{y}^{(\alpha)}$ 定义中归一化因子的求和形式，SGD算法效率将很低，因此我们换用另一种思路：

<strong>将整个vocabulary上的softmax问题，看做是对vocabulary中每个词的二元分类问题</strong>（$w_i$ is or not the target of given context $c^{(\alpha)}$），于是对 $\forall$ sample $\alpha$ 和 $\forall i=0,1,\dots V-1$，定义2维的归一化矢量：

$$\hat{y}_i^{(\alpha)}=\begin{bmatrix}
\sigma\left(C^{(\alpha)}\cdot W_i\right)\\
1-\sigma\left(C^{(\alpha)}\cdot W_i\right)\\
\end{bmatrix}\quad\text{用来描述 target of sample $\alpha$ 是$w_i$的概率预期},\quad \sigma(z)\equiv {1\over 1+ e^{-z}}$$

根据 target of sample $\alpha$的经验值, $t^{(\alpha)}=w_p$ ，我们可以对vocabulary中的每个词定义2维one-hot矢量

$$y_i^{(\alpha)} = \left\{\begin{matrix}
[1, 0]&\quad i=p&, \text{即$w_i$ 是 target of context $c^{(\alpha)}$ 的经验值}\\
[0, 1]&\quad i\ne p&
\end{matrix}\right.$$

相应的cross-entropy函数为

$$\mathscr{C}^{(\alpha)}=-\sum\limits_i y_i^{(\alpha)}\cdot \log \hat{y}_i^{(\alpha)} = -\left( \log \sigma_p^{(\alpha)}  + \sum\limits_{i\ne p} \left(1-\log \sigma_i^{(\alpha)}\right) \right),\quad \sigma_i^{(\alpha)}\equiv\sigma\left(C^{(\alpha)}\cdot W_i\right) $$

在实际计算中，$\sum\limits_{i\ne p}$ 项我们将仅根据一定分布随机采样$k$个非$p$值进行求和,称为<strong> negative sampling</strong>,即

$$\mathscr{C}^{(\alpha)} = -\left( \log \sigma_p^{(\alpha)}  + \sum\limits_{i\in \mathcal{N_k}} \left(1-\log \sigma_i^{(\alpha)}\right) \right),\quad \mathcal{N_k}\text{表示含有$k$个元素的negative sample集合} $$

同样整个样本集$\mathcal{D}$ 的cost function 为：

$$\mathscr{C} = {1\over N}\sum\limits_{\alpha=1}^N\mathscr{C}^{(\alpha)}$$

vocabulary 中每个词对应的 embedding vector为：

$$\left(\;W_0^*, W_1^*,\dots, W_{V-1}^*\;\right) ={\rm argmin} \mathscr{C}\left(\;W_0, W_1,\dots, W_{V-1}\;\right)$$


# unidirectional RNNs

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/

http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/

http://www.wildml.com/2015/10/recurrent-neural-network-tutorial-part-4-implementing-a-grulstm-rnn-with-python-and-theano/

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/





给定一个长度为 $t$ 的sentence，用一个序列 $[x_0, x_1, \dots, x_{t-1}]$ 来表征，其中每个 $x_i$ 是一个$N$维向量，表征一个word，hidden state与此输入序列对应的$t$个“反馈值”记为 $[h_0, h_1, \dots, h_{t-1}]$, 每个 $h_i$ 是一个$D$维向量（$D$ 是hidden state的维度）, 并令 $h_{-1} = \underbrace{[0,0,\dots,0]}_{D \text{个}}$ （或其他某个值，表征initial hidden state），hidden state“反馈值”的决定方法如下：

<table>
    <tr><td style="text-align:center">$h_{-1}=0$</td></tr>
    <tr>
        <td>$\begin{matrix}
        h_0&=&H\left(h_{-1},\;x_0,\;\Theta_H\right)\\
        h_1&=&H\left(h_0,\;x_1,\;\Theta_H\right)\\
        &\vdots&\\
        h_{t-2}&=&H\left(h_{t-3},\;x_{t-2},\;\Theta_H\right)\\
        h_{t-1}&=&H\left(h_{t-2},\;x_{t-1},\;\Theta_H\right)
        \end{matrix}$</td>
    </tr>
</table>

表中$H(\cdots)$ 表示 hidden state 的决定函数，其中$\Theta_H$ 是该函数的所有参数



 ### 1. sentence 的 情感分类问题

在情感分类问题中，输出 $\hat{y}$ 通常是一个softmax产生的$K$维矢量，表征sentence所属情感类别的概率分布 （$K$ 是情感类别数目），如下表：


<table>
    <tr><td style="text-align:center">$h_{-1}=0$</td></tr>
    <tr>
        <td>$\begin{matrix}
        h_0&=&H\left(h_{-1},\;x_0,\;\Theta_H\right)\\
        h_1&=&H\left(h_0,\;x_1,\;\Theta_H\right)\\
        &\vdots&\\
        h_{t-2}&=&H\left(h_{t-3},\;x_{t-2},\;\Theta_H\right)
        \end{matrix}$</td>   
    </tr>
     <tr>
        <td>$\begin{matrix}
        h_{t-1}&=&H\left(h_{t-2},\;{x}_{t-1},\;\Theta_H\right)
        \end{matrix}$</td>
        <td>$\begin{matrix}
        \hat{y}&=&Y\left(h_{t-1},\;\Theta_Y\right)
        \end{matrix}$</td>    
    </tr>
</table>


其中 $Y(\cdots)$ 表示 output 的决定函数，其中$\Theta_Y$ 是该函数的所有参数。

<strong>怎样训练</strong>:

根据该sentence已标注的类别，可以定义 $K$ 维 one-hot 矢量 $y$ （$K$ 为类别总数），并用cross entropy 作为 cost function：

$$\mathscr{C}\equiv -y\cdot\log \hat{y}$$

一个语料库，通常分割为很多sentence (sentence 数目记为$M$), 则总的cost function 为:

$$\mathscr{C}\equiv -{1\over M}
\left(\sum\limits_{i=1}^{M}y^{(i)}\cdot\log \hat{y}^{(i)}\right)
$$


于是 training 的最优化问题，可以表示为寻找最优参数 $\Theta_H^*,\Theta_Y^*$：

$$\left(\Theta_H^*,\Theta_Y^*\right) = {\rm argmin} \mathscr{C}\left(\Theta_H,\Theta_Y\right)$$

<strong>怎样预测一个sentence的情感分类</strong>:

(略)

### 2. 自动生成sentence的问题

在文本自动生成的问题中，每一步输入$x_i$, 都对应一个输出$\hat{y}_i$, $\hat{y}_i$ 通常是一个softmax产生的$K$维的矢量，表征$x_i$ 下一个词可能性的概率分布 （$K$ 是 vocabulary size），如下表：


<table>
    <tr><td style="text-align:center">$h_{-1}=0$</td></tr>
    <tr>
        <td>$\begin{matrix}
        h_0&=&H\left(h_{-1},\;x_0,\;\Theta_H\right)\\
        h_1&=&H\left(h_0,\;x_1,\;\Theta_H\right)\\
        &\vdots&\\
        h_{t-2}&=&H\left(h_{t-3},\;x_{t-2},\;\Theta_H\right)\\
        h_{t-1}&=&H\left(h_{t-2},\;x_{t-1},\;\Theta_H\right)
        \end{matrix}$</td>
        <td>$\begin{matrix}
        \hat{y}_0&=&Y\left(h_0,\Theta_Y\right)\\
        \hat{y}_1&=&Y\left(h_1,\Theta_Y\right)\\
        &\vdots&\\
        \hat{y}_{t-2}&=&Y\left(h_{t-2},\Theta_Y\right)\\
        \hat{y}_{t-1}&=&Y\left(h_{t-1},\Theta_Y\right)\\
        \end{matrix}$</td>
    </tr>
</table>

其中 $Y(\cdots)$ 表示 output 的决定函数，其中$\Theta_Y$ 是该函数的所有参数。

<strong>generative procedure 依据的概率解释</strong>:

$\hat{y}_i$ 表示 给定序列 $[x_0,x_1,\dots,x_i]$ 后，下一个词在vacalulary中的<strong>条件</strong>概率 , 即$\hat{y}_i$的概率意义可以由如下的Bayes公式体现：

$$\begin{matrix}
P([x_0,x_1,\dots,x_{i}]) &=& P(x_0)&P(x_1 | [x_0])&P(x_2| [x_0, x_1])&\cdots &P(x_i| [x_0, x_1,\cdots,x_{i-1}])\\
&&\downarrow&\downarrow&\downarrow&\cdots &\downarrow\\
&&\hat{y}_0&\hat{y}_1&\hat{y}_2&\cdots &\hat{y}_i
\end{matrix}$$
<strong>怎样训练</strong>:

根据该sentence中真实的后续词，可以定义 $t$个 $K$ 维 one-hot 矢量 $y_i$ （$t$ 为sentence length， $K$ 为vocabulary size），并用cross entropy 作为 cost function：

$$\mathscr{C}\equiv -{1\over t} \sum\limits_{j=0}^{t-1}y_j\cdot\log \hat{y}_j$$

一个语料库，通常分割为很多sentence (sentence 数目记为$M$), 则总的cost function 为:

$$\mathscr{C}\equiv -{1\over t_1 + t_2+\dots + t_M}\left[
\left(\sum\limits_{j=0}^{t_1-1}y_j^{(1)}\cdot\log \hat{y}_j^{(1)}\right)+
\left(\sum\limits_{j=0}^{t_2-1}y_j^{(2)}\cdot\log \hat{y}_j^{(2)}\right)+
\dots+
\left(\sum\limits_{j=0}^{t_M-1}y_j^{(M)}\cdot\log \hat{y}_j^{(M)}\right)
\right]$$


其中$t_i$表示第$i$个sentence的长度。于是 training 的最优化问题，可以表示为寻找最优参数 $\Theta_H^*,\Theta_Y^*$：

$$\left(\Theta_H^*,\Theta_Y^*\right) = {\rm argmin} \mathscr{C}\left(\Theta_H,\Theta_Y\right)$$

<strong>怎样自动生成一个sentence</strong>：

给定sentence的若干个初始词（数目记为$n$个） $x_0,x_1,\dots,x_{n-1}$，


<table>
    <tr><td style="text-align:center">$h_{-1}=0$</td></tr>
    <tr>
        <td>$\begin{matrix}
        h_0&=&H\left(h_{-1},\;x_0,\;\Theta_H\right)\\
        h_1&=&H\left(h_0,\;x_1,\;\Theta_H\right)\\
        &\vdots&\\
        h_{n-2}&=&H\left(h_{n-3},\;x_{n-2},\;\Theta_H\right)
        \end{matrix}$</td>   
    </tr>
     <tr>
        <td>$\begin{matrix}
        h_{n-1}&=&H\left(h_{n-2},\;{x}_{n-1},\;\Theta_H\right)
        \end{matrix}$</td>
        <td>$\begin{matrix}
        \hat{y}_{n-1}&=&Y\left(h_{n-1},\;\Theta_Y\right)
        \end{matrix}$</td>
        <td>$\begin{matrix}
        \text{下个词取$\hat{y}_{n-1}$中概率最大的词，其向量记为${x}_n\quad$}
        \end{matrix}$</td>     
    </tr>
    <tr>
        <td>$\begin{matrix}
        h_{n}&=&H\left(h_{n-1},\;{x}_{n},\;\Theta_H\right)
        \end{matrix}$</td>
        <td>$\begin{matrix}
        \hat{y}_{n}&=&Y\left(h_{n},\;\Theta_Y\right)
        \end{matrix}$</td>
        <td>$\begin{matrix}
        \text{再下个词取$\hat{y}_{n}$中概率最大的词，其向量记为${x}_{n+1}\quad$}
        \end{matrix}$</td>     
    </tr>
    <tr><td colspan="3" style="text-align:center">$\vdots$</td></tr>
    <tr><td colspan="3" style="text-align:center">如此反复，直至生成的词，对应句子的终止符</td></tr>

</table>

### 3. 翻译sentence的问题


# Bidirectional RNNs



<table>
    <tr><td style="text-align:center">$h_{-1}=0$</td><td></td></tr>
    <tr>
        <td>$\begin{matrix}
        H\left(h_{-1},\;x_0,\;\Theta_H\right)&=&h_0\\
        H\left(h_0,\;x_1,\;\Theta_H\right)&=&h_1\\
        &\vdots&\\
        H\left(h_{t-3},\;x_{t-2},\;\Theta_H\right)&=&h_{t-2}\\
        H\left(h_{t-2},\;x_{t-1},\;\Theta_H\right)&=&h_{t-1}\\
        \end{matrix}$</td>
        <td>$\begin{matrix}
        G\left(g_{1},\;x_0,\;\Theta_G\right)&=&g_0\\
        G\left(g_{2},\;x_1,\;\Theta_G\right)&=&g_1\\
        &\vdots&\\
        G\left(g_{t-1},\;x_{t-2},\;\Theta_G\right)&=&g_{t-2}\\
        G\left(g_{t},\;x_{t-1},\;\Theta_G\right)&=&g_{t-1}\\
        \end{matrix}$</td>
        <td>$\begin{matrix}
        Y\left(h_0,\;g_0,\;\Theta_Y\right)&=&y_0\\
        Y\left(h_1,\;g_1,\;\Theta_Y\right)&=&y_1\\
        &\vdots&\\
        Y\left(h_{t-2},\;g_{t-2},\;\Theta_Y\right)&=&y_{t-2}\\
        Y\left(h_{t-1},\;g_{t-1},\;\Theta_Y\right)&=&y_{t-1}\\
        \end{matrix}$</td>
    </tr>
    <tr><td></td><td style="text-align:center">$g_{t}=0$</td></tr>
</table>

### 1. 填空问题


<table>
<tr>
<td colspan=2 rowspan=2>$\;P_t(x=1)\quad$</td>
<td colspan=2>$X_{t-8}$</td>
</tr>
<tr>
<td>1</td>
<td>0</td>
</tr>
<tr>
<td rowspan=2>$X_{t-3}$</td>
<td>1</td>
<td>0.75</td>
<td>1</td>
</tr>
<tr>
<td>0</td>
<td>0.25</td>
<td>0.5</td>
</tr>
</table>

[CNN_LSTM.pdf](../papers/CNN_LSTM.pdf)
